In [1]:
import requests
from bs4 import BeautifulSoup
import json

base_url = "https://luatvietnam.vn/an-ninh-trat-tu/nghi-dinh-168-2024-nd-cp-xu-phat-vi-pham-hanh-chinh-giao-thong-duong-bo-tru-diem-phuc-hoi-diem-giay-phep-lai-xe-382883-d1.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
}

response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

chapters = []
current_chapter = None
current_text = ''

docitems = soup.find_all("div", class_=lambda x: x and x.startswith("docitem"))
docitems = docitems[4:]  # Skip header elements

for div in docitems:
    class_name = div.get('class')[0]
    
    if class_name == 'docitem-2':
        # Save previous chapter's content
        if current_chapter and current_text:
            current_chapter['content'].append(current_text)
        
        # Start new chapter
        if current_chapter:
            chapters.append(current_chapter)
        current_chapter = {
            'title': div.text.strip(),
            'content': []
        }
        current_text = ''
        
    elif class_name == 'docitem-5':
        # Save previous section if exists
        if current_text:
            current_chapter['content'].append(current_text)
        # Start new section
        current_text = div.text.strip()
        
    elif class_name in ['docitem-11', 'docitem-12']:
        # Append to current section
        current_text += '\n' + div.text.strip()

# Save final chapter and its content
if current_chapter and current_text:
    current_chapter['content'].append(current_text)
if current_chapter:
    chapters.append(current_chapter)

# Save to JSON file
with open('chapters.json', 'w', encoding='utf-8') as f:
    json.dump(chapters, f, ensure_ascii=False, indent=2)

print(f"Saved {len(chapters)} chapters to chapters.json")

Saved 4 chapters to chapters.json


In [1]:
import requests
from bs4 import BeautifulSoup
import json

base_url = "https://luatvietnam.vn/an-ninh-trat-tu/nghi-dinh-168-2024-nd-cp-xu-phat-vi-pham-hanh-chinh-giao-thong-duong-bo-tru-diem-phuc-hoi-diem-giay-phep-lai-xe-382883-d1.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
}

response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

data = []
current_chapter = None
current_text = ''

docitems = soup.find_all("div", class_=lambda x: x and x.startswith("docitem"))
docitems = docitems[5:] 

In [2]:
len(docitems)

2059

In [3]:
doc_5 = soup.find_all("div", class_="docitem-5")

In [4]:
doc_5

[<div class="docitem-5" data-id="1205805" id="demuc1205805"><b>Điều 1. Phạm vi điều chỉnh</b></div>,
 <div class="docitem-5" data-id="1205808" id="demuc1205808"><b>Điều 2. Đối tượng áp dụng</b></div>,
 <div class="docitem-5" data-id="1205822" id="demuc1205822"><b>Điều 3. Giải thích từ ngữ</b><br/>
 Trong Nghị định này, các từ ngữ dưới đây được hiểu như sau:</div>,
 <div class="docitem-5" data-id="1205840" id="demuc1205840"><div class="popupRelate docitem-47" data-href="/noi-dung-thay-doi.html?DocItemId=1205840&amp;DocItemRelateId_Select=99637" data-title="&lt;span class='tooltip-47' &gt;Điều 4a được bổ sung bởi Khoản 2 Điều 2 Nghị định số 123/2021/NĐ-CP&lt;/span&gt;"><b>Điều 4. Các biện pháp khắc phục hậu quả và nguyên tắc áp dụng</b></div></div>,
 <div class="docitem-5" data-id="1205899" id="demuc1205899"><b>Điều 5. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy tắc giao thông đường bộ</b></div>,
 <div class="docitem-5" data-id="1205985" id="demuc1205985"

In [5]:
indices = []
for value in doc_5:
    try:
        index = docitems.index(value)
        indices.append(index)
    except ValueError:
        indices.append(None)  # or handle the missing value as needed

print(indices)

[0, 3, 18, 36, 98, 191, 283, 345, 389, 397, 412, 439, 485, 507, 514, 556, 592, 619, 624, 644, 655, 688, 696, 761, 810, 825, 832, 835, 932, 938, 1056, 1067, 1076, 1098, 1109, 1124, 1135, 1196, 1220, 1233, 1251, 1269, 1282, 1297, 1309, 1315, 1323, 1337, 1363, 1394, 1400, 1428, 1441, 1462, 1479, 1503, 1529, 1537, 1548, 1553, 1567, 1582, 1586, 1593, 1603, 1616, 1646, 1654, 1661, 1667, 1678, 1697, 1712, 1729, 1832, 1851, 1882, 1916, 1920, 1932, 1988, 2011, 2032, 2053, 2056, 2057]


In [83]:
dataset = []

In [71]:
docitems[95]

<div class="popupRelate docitem-58" data-href="/noi-dung-thay-doi.html?DocItemId=1205840&amp;DocItemRelateId_Select=99637" data-title="&lt;span class='tooltip-58'&gt;Điều 4a được bổ sung bởi Khoản 2 Điều 2 Nghị định số 123/2021/NĐ-CP&lt;/span&gt;">Bổ sung</div>

In [102]:
docitems[18:36]

[<div class="docitem-5" data-id="1205822" id="demuc1205822"><b>Điều 3. Giải thích từ ngữ</b><br/>
 Trong Nghị định này, các từ ngữ dưới đây được hiểu như sau:</div>,
 <div class="docitem-11" data-id="1205823">1. Lĩnh vực giao thông đường bộ:</div>,
 <div class="docitem-12" data-id="1205824">a) Máy kéo là loại xe gồm phần đầu máy tự di chuyển, được lái bằng càng hoặc vô lăng và rơ moóc được kéo theo (có thể tháo rời với phần đầu kéo);</div>,
 <div class="docitem-12" data-id="1205825">b) Các loại xe tương tự xe ô tô là loại phương tiện giao thông đường bộ chạy bằng động cơ có từ hai trục, bốn bánh xe trở lên, có phần động cơ và thùng hàng (nếu có) lắp trên cùng một xát xi (kể cả loại xe 4 bánh chạy bằng năng lượng điện);</div>,
 <div class="docitem-12" data-id="1205826">c) Các loại xe tương tự xe mô tô là phương tiện giao thông đường bộ chạy bằng động cơ, có hai bánh hoặc ba bánh, có dung tích làm việc của động cơ từ 50 cm3 trở lên, có vận tốc thiết kế lớn nhất lớn hơn 50 km/h, có khối l

In [270]:
data_instruct = docitems[98].text.strip()
print(data_instruct)
print("*"*50)

output_content=""
for i in range(189,191):
    print(docitems[i].text.strip())
    print("-"*50)
    output_content = output_content + docitems[i].text.strip()
    


Điều 5. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy tắc giao thông đường bộ
**************************************************
g) Thực hiện hành vi quy định tại điểm c khoản 8 Điều này bị tước quyền sử dụng Giấy phép lái xe từ 16 tháng đến 18 tháng;
--------------------------------------------------
h) Thực hiện hành vi quy định tại khoản 10 Điều này bị tước quyền sử dụng Giấy phép lái xe từ 22 tháng đến 24 tháng.
--------------------------------------------------


In [271]:
dataset.append({
        "instruct":data_instruct,
        "output":output_content
    })

In [195]:
dataset

[{'instruct': 'Điều 1. Phạm vi điều chỉnh',
  'output': '1. Nghị định này quy định về hành vi vi phạm hành chính; hình thức, mức xử phạt, biện pháp khắc phục hậu quả đối với từng hành vi vi phạm hành chính; thẩm quyền lập biên bản, thẩm quyền xử phạt, mức phạt tiền cụ thể theo từng chức danh đối với hành vi vi phạm hành chính trong lĩnh vực giao thông đường bộ và đường sắt.2. Các hành vi vi phạm hành chính trong các lĩnh vực quản lý nhà nước khác liên quan đến lĩnh vực giao thông đường bộ và đường sắt mà không quy định tại Nghị định này thì áp dụng quy định tại các Nghị định quy định về xử phạt vi phạm hành chính trong các lĩnh vực đó để xử phạt.'},
 {'instruct': 'Điều 2. Đối tượng áp dụng',
  'output': '1. Cá nhân, tổ chức có hành vi vi phạm hành chính trong lĩnh vực giao thông đường bộ, đường sắt trên lãnh thổ nước Cộng hòa xã hội chủ nghĩa Việt Nam.2. Tổ chức quy định tại khoản 1 Điều này gồm:a) Cơ quan nhà nước có hành vi vi phạm mà hành vi đó không thuộc nhiệm vụ quản lý nhà nước 

In [272]:
import json

json_output = json.dumps(dataset, ensure_ascii=False, indent=4)
print(json_output)


[
    {
        "instruct": "Điều 1. Phạm vi điều chỉnh",
        "output": "1. Nghị định này quy định về hành vi vi phạm hành chính; hình thức, mức xử phạt, biện pháp khắc phục hậu quả đối với từng hành vi vi phạm hành chính; thẩm quyền lập biên bản, thẩm quyền xử phạt, mức phạt tiền cụ thể theo từng chức danh đối với hành vi vi phạm hành chính trong lĩnh vực giao thông đường bộ và đường sắt.2. Các hành vi vi phạm hành chính trong các lĩnh vực quản lý nhà nước khác liên quan đến lĩnh vực giao thông đường bộ và đường sắt mà không quy định tại Nghị định này thì áp dụng quy định tại các Nghị định quy định về xử phạt vi phạm hành chính trong các lĩnh vực đó để xử phạt."
    },
    {
        "instruct": "Điều 2. Đối tượng áp dụng",
        "output": "1. Cá nhân, tổ chức có hành vi vi phạm hành chính trong lĩnh vực giao thông đường bộ, đường sắt trên lãnh thổ nước Cộng hòa xã hội chủ nghĩa Việt Nam.2. Tổ chức quy định tại khoản 1 Điều này gồm:a) Cơ quan nhà nước có hành vi vi phạm mà hành v

In [273]:
file_name = "Dung_o_dieu_5.json"
try:
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(json_output)
    print(f"Data successfully saved to {file_name}")
except Exception as e:
    print(f"Error saving file: {e}")

Data successfully saved to Dung_o_dieu_5.json
